# Example 1: Constructing the reduced Hessian

In this example, we will solve a model and check the eigenvalues of the reduced Hessian at the solution. The purpose of this example is to (a) practice using `PyomoNLP` and (b) introduce you to the reduced Hessian, which can be useful for debugging regularization coefficients.

We start with some imports.

In [ ]:
import pyomo.environ as pyo
import pyomo.environ as pyo
from pyomo.common.collections import ComponentSet
from pyomo.contrib.incidence_analysis import IncidenceGraphInterface
from pyomo.contrib.pynumero.interfaces.pyomo_nlp import PyomoNLP
from svi.auto_thermal_reformer.fullspace_flowsheet import make_optimization_model
import numpy as np
import scipy as sp

The model we will solve comes from the "surrogate-vs-implicit" repository, available [here](https://github.com/robbybp/surrogate-vs-implicit). We have imported this package above, and will construct an autothermal reforming flowsheet model using its function `make_optimization_model`.

In [ ]:
def make_and_solve_model(**kwds):
    # First, we make our model
    P = kwds.pop("P", 1.5e6)
    # With X = 0.95, we converge. With X = 0.94, we don't. Go figure
    X = kwds.pop("X", 0.94)
    m = make_optimization_model(X, P)
    m.ipopt_zL_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.ipopt_zU_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)
    # Does the model solve?
    solver = pyo.SolverFactory("cyipopt", options=kwds)
    res = solver.solve(m, tee=True)
    # Converges infeasible in 900 iter
    return m, res

In [ ]:
m, res = make_and_solve_model(X=0.95)
pyo.assert_optimal_termination(res)

Now we'll construct a `PyomoNLP` at the solution. This will give us access to the derivative matrices we'll need to construct the reduced Hessian.

In [ ]:
nlp = PyomoNLP(m)

The reduced Hessian relies on knowledge of our "degrees of freedom." We'll assume we know what these are.

In [ ]:
dof_varnames = [
    "fs.reformer_bypass.split_fraction[0.0,bypass_outlet]",
    "fs.reformer_mix.steam_inlet_state[0.0].flow_mol",
    "fs.feed.properties[0.0].flow_mol",
]
dofvars = [m.find_component(name) for name in dof_varnames]
dofvar_set = ComponentSet(dofvars)
basicvars = [v for v in nlp.get_pyomo_variables() if v not in dofvar_set]
print(f"N. dof variables:   {len(dofvars)}")
print(f"N. basic variables: {len(basicvars)}")

We need the Jacobian of our equality constraints with respect to "basic variables" to be nonsingular.

In [ ]:
eqcons = nlp.get_pyomo_equality_constraints()
print(f"N. equality constraints: {len(eqcons)}")
basic_submatrix = nlp.extract_submatrix_jacobian(basicvars, eqcons)
try:
    lu = sp.sparse.linalg.splu(basic_submatrix.tocsc())
    print(f"Eq. Jac. has rank at least {len(basicvars)}")
except RuntimeError:
    # This matrix being singular doesn't prove anything about the rank.
    # We could have just chosen a bad set of degrees of freedom.
    print("Eq. Jac. is not (necessarily) full row rank")

This proves that our equality Jacobian is full row rank. Now we can start worrying about the reduced Hessian.
When constructing anything involving the Hessian of the Lagrangian, it's always a good idea to construct the
gradient of the Lagrangian and make sure we have the right sign convention.

In [ ]:
def get_gradient_of_lagrangian(
    nlp,
    primal_lb_multipliers,
    primal_ub_multipliers,
):
    grad_obj = nlp.evaluate_grad_objective()
    jac = nlp.evaluate_jacobian()
    duals = nlp.get_duals()
    conjac_term = jac.transpose().dot(duals)
    grad_lag = (
        - grad_obj
        - conjac_term
        + primal_lb_multipliers
        - primal_ub_multipliers
    )
    return grad_lag

lbmult = [m.ipopt_zL_out[x] for x in nlp.get_pyomo_variables()]
ubmult = [m.ipopt_zU_out[x] for x in nlp.get_pyomo_variables()]
gradlag = get_gradient_of_lagrangian(nlp, lbmult, ubmult)
print(max(abs(gradlag)))

Looks good. Now we can start working on the reduced Hessian. The reduced Hessian is the Hessian of the Lagrangian projected onto the null space of the equality constraint Jacobian. We'll start by constructing the Hessian of the Lagrangian, with variables in the order `(dofvars, basicvars)`.

In [ ]:
varorder = dofvars + basicvars
varindices = nlp.get_primal_indices(varorder)
hess = nlp.extract_submatrix_hessian_lag(varorder, varorder)

This is the Hessian of the Lagrangian, but we're still missing terms for the bounds and inequalities, i.e., $\Sigma_x = X^{-1}V$, where $X$ is the difference between $X$ and its bound and $V$ is the diagonal matrix of bound multipliers. The term for inequalities is $\nabla_x g^T G^{-1} \Lambda_g \nabla_x g$, which we get by pivoting the KKT matrix on $G$, the diagonal matrix of inequality constraint values. ($\Lambda_g$ is the diagonal matrix of inequality constraint multipliers.)

In [ ]:
# First we'll get the bound terms
lbmult = sp.sparse.diags([m.ipopt_zL_out[v] for v in nlp.get_pyomo_variables()])
ubmult = sp.sparse.diags([m.ipopt_zU_out[v] for v in nlp.get_pyomo_variables()])
primal_val = nlp.get_primals()[varindices]
primal_lb = nlp.primals_lb()[varindices]
primal_ub = nlp.primals_ub()[varindices]
active_lbs = np.where(primal_lb == primal_val)[0]
active_ubs = np.where(primal_ub == primal_val)[0]
# Push values into interior if bounds are exactly active, so we don't divide by zero
primal_val[active_lbs] += 1e-8
primal_val[active_ubs] -= 1e-8
primal_ubslack_inv = sp.sparse.diags(1 / (primal_ub - primal_val))
primal_lbslack_inv = sp.sparse.diags(1 / (primal_val - primal_lb))
bound_term = lbmult @ primal_ubslack_inv + ubmult @ primal_lbslack_inv

# Now we construct the inequality term
ineqcons = nlp.get_pyomo_inequality_constraints()
ineq_jac = nlp.extract_submatrix_jacobian(varorder, ineqcons)
ineq_val = nlp.evaluate_ineq_constraints()
ineq_val_inv = sp.sparse.diags(1 / ineq_val)
ineq_duals = sp.sparse.diags(nlp.get_duals_ineq())
ineq_term = ineq_jac.transpose() @ ineq_val_inv @ ineq_duals @ ineq_jac

# Now we add these to our Hessian
hess += bound_term
# We subtract the inequality term as our inequalities are `g(x) <= 0` (whereas we have
# written the bound constraints as ">=" constraints, e.g., `x - x^L >= 0`).
hess -= ineq_term

Now we can construct the reduced Hessian. Recall that the reduced Hessian is the Hessian of the Lagrangian projected onto the null space of the equality constraint Jacobian, i.e., $Z^T \mathcal{H} Z$, where
$Z$ is a basis for the equality Jacobian's null space. Null space bases are not unique, but we can get
*a* null space basis from the partition of our variables into "degrees of freedom" and "basic." See [here](https://epubs.siam.org/doi/10.1137/0607059) for more information on the problem of computing a null space basis
of a sparse matrix.

In [ ]:
dof_submatrix = nlp.extract_submatrix_jacobian(dofvars, eqcons)
ndof = len(dofvars)
dof_nullbasis = np.identity(ndof)
lu = sp.sparse.linalg.splu(basic_submatrix.tocsc())
basic_nullbasis = - lu.solve(dof_submatrix.toarray())
nullbasis = np.vstack((dof_nullbasis, basic_nullbasis))
rh = nullbasis.transpose() @ hess @ nullbasis
print("Reduced Hessian:")
print(rh)

We have the reduced Hessian. Now let's check its eigenvalues.

In [ ]:
evals, evecs = np.linalg.eig(rh)
print(evals)

We have one (slightly) negative eigenvalue. Is that a problem? Your mileage may vary. Inertia, especially of small eigenvalues, can be fairly sensitive to the method used to compute it.

The reduced Hessian is an important object in the theory of nonlinear optimization, mostly for ensuring that
we are taking a descent direction.
The magnitude of the eigenvalues here tells us that this problem is poorly scaled (with respect to these three degrees of freedom &mdash; maybe another choice is better scaled).
Constructing this matrix can be useful for debugging large regularization coefficients that you might see when
solving with Ipopt.
If you can confirm that these regularization coefficients are caused by a large, negative eigenvalue in the reduced
Hessian, you can potentially correct it by adding curvature in this eigenvalue's coordinate (say, by adding a bound on the corresponding variable).

That's the end of this tutorial! I encourage you to copy the code in the `example1.py` file and try this out on your own nonlinear optimization problems. 